In [1]:
#%reset 
# clears variables in workspace
# note on resetting:
# especially while you're debugging, you want to clear your variables between attempts to solve. Why?
# A) you may have noticed that pyomo doesn't like things being redefined
# more importantly, B) the variables keep their values, so an infeasible solution becomes an infeasible guess. 

# import libraries
# Pyomo stuff
from pyomo.environ import*
from pyomo.opt import SolverFactory
from pyomo.opt import SolverStatus, TerminationCondition
import numpy as np

from IPython.display import display #for pretty printing

# create the model
m = ConcreteModel()

# SETS-----------------------------------------------------------------------------------------------------------------------

#N = 100
N = 30
m.N = RangeSet(N) 

# sets can have multidimensional indices
# (probably a little gratuitous for such a simple model, but thought I'd show you anyway)
links = [('femur',1),('tibia',1),('femur',2),('tibia',2)]
m.L = Set(dimen=2, initialize = links)

DOFs = ['x','y','theta_hip_1','theta_knee_1','theta_hip_2','theta_knee_2']
m.DOF = Set(initialize = DOFs) # the coordinates for each link

In [2]:
# PARAMETERS-----------------------------------------------------------------------------------------------------------------

m.g = Param(initialize = 9.81)

# mass of links
def get_m(n, lb, ln):
    if lb == 'femur':
        return 0.35
    else: return 0.15
m.m = Param(m.L, initialize = get_m)

mbody = sum(m.m[l] for l in links)
mBW = mbody*m.g.value

# length of links
def get_len(n, lb, ln):
    if lb == 'femur':
        return 0.12
    else: return 0.24
m.len = Param(m.L, initialize = get_len) 

#Inertia
def calculate_In(m, lb, ln): 
    l = (lb,ln)
    # yes, that does mean you have to rebuild the tuple inside the function. Yes, it is dumb.
    return m.m[l]*m.len[l]**2/12 
m.In = Param(m.L, initialize = calculate_In) # moment of inertia

# VARIABLES -----------------------------------------------------------------------------------------------------------------

# system coordinates
m.X = Var(m.N, m.L, m.DOF) # position
m.dX = Var(m.N, m.L, m.DOF) # velocity
m.ddX = Var(m.N, m.L, m.DOF) # acceleration

# bound variables
for n in range(1,N+1):
    for l in links:
        m.X[n,l,'y'].setlb(0.0)
    
print("Done")

Done


# Contacts

In [3]:
# variable timestep
hm = 0.02 # master timestep
m.h = Var(m.N, bounds = (0.8,1.2))

# Integration constraints 
def BwEuler_p(m,n,lb,ln,dof): # for positions
    l = (lb,ln) # why. whyyyyyy.
    if n > 1:
        return m.X[n,l,dof] == m.X[n-1,l,dof] + hm*m.h[n]*m.dX[n,l,dof]
    else:
        return Constraint.Skip #use this to leave out members of a set that the constraint doesn't apply to
m.integrate_p = Constraint(m.N, m.L, m.DOF, rule = BwEuler_p)

def BwEuler_v(m,n,lb,ln,dof): # for velocities
    l = (lb,ln) #ffs
    if n > 1:
        return m.dX[n,l,dof] == m.dX[n-1,l,dof] + hm*m.h[n]*m.ddX[n-1,l,dof]
    else:
        return Constraint.Skip 
m.integrate_v = Constraint(m.N, m.L, m.DOF, rule = BwEuler_v)

print("Done")

Done


In [4]:
# GROUND INTERACTIONS -------------------------------------------------------------------------------------------------------

# paramters
m.mu = Param(initialize = 1) # friction coefficient

# sign set for positive and negative components
signs = ['ps','ng'] 
m.sgn = Set(initialize = signs)

# variables
# (I had that set error while trying to set up these default bounds. That's why I pointed it out.)

m.foot1_p = Var(m.N, m.DOF, bounds = (0.0,None)) # foot position
m.foot1_v = Var(m.N, m.DOF, m.sgn, bounds = (0.0,None)) # foot velocity
m.foot2_p = Var(m.N, m.DOF, bounds = (0.0,None)) # foot position
m.foot2_v = Var(m.N, m.DOF, m.sgn, bounds = (0.0,None)) # foot velocity

m.friction_cone1 = Var(m.N, bounds = (0.0,None))
m.friction_cone2 = Var(m.N, bounds = (0.0,None))

m.GRF1 = Var(m.N, m.DOF, m.sgn, bounds = (0.0,None)) # ground reaction forces
m.GRF2 = Var(m.N, m.DOF, m.sgn, bounds = (0.0,None)) # ground reaction forces

ground_constraints = ['contact','friction','slip_ps','slip_ng'] 
m.ground_constraints1 = Set(initialize = ground_constraints) # set for indexing ground-related penalties
m.ground_penalty1 = Var(m.N, m.ground_constraints1, bounds = (0.0,None))
m.ground_constraints2 = Set(initialize = ground_constraints) # set for indexing ground-related penalties
m.ground_penalty2 = Var(m.N, m.ground_constraints2, bounds = (0.0,None))

# constraints: aux variables

def def_foot1_v(m,n,dof):
    if dof == 'x':
        return m.foot1_v[n,dof,'ps']-m.foot1_v[n,dof,'ng'] == m.dX[n,('tibia',1),'x'] + m.dX[n,('tibia',1),'theta_knee_1']*0.5*m.len[('tibia',1)]*cos(m.X[n,('tibia',1),'theta_knee_1'])
    else:
        return Constraint.Skip
m.def_foot1_v = Constraint(m.N, m.DOF, rule = def_foot1_v)

def def_foot2_v(m,n,dof):
    if dof == 'x':
        return m.foot2_v[n,dof,'ps']-m.foot2_v[n,dof,'ng'] == m.dX[n,('tibia',2),'x'] + m.dX[n,('tibia',2),'theta_knee_2']*0.5*m.len[('tibia',2)]*cos(m.X[n,('tibia',2),'theta_knee_2'])
    else:
        return Constraint.Skip
m.def_foot2_v = Constraint(m.N, m.DOF, rule = def_foot2_v)

def def_friction_cone1(m,n):
    return m.friction_cone1[n] == m.mu*m.GRF1[n,'y','ps'] - (m.GRF1[n,'x','ps'] + m.GRF1[n,'x','ng'])
m.def_friction_cone1 = Constraint(m.N, rule = def_friction_cone1)

def def_friction_cone2(m,n):
    return m.friction_cone2[n] == m.mu*m.GRF2[n,'y','ps'] - (m.GRF2[n,'x','ps'] + m.GRF2[n,'x','ng'])
m.def_friction_cone2 = Constraint(m.N, rule = def_friction_cone2)

# constraints: complementarity

# contact
def ground_contact1(m,n):
    if n < N:
        return m.ground_penalty1[n,'contact'] == m.foot1_p[n+1,'y']*m.GRF1[n,'y','ps'] 
        # notice that the GRF is complemented with the foot height at the NEXT node
    else:
        return Constraint.Skip
m.ground_contact1 = Constraint(m.N, rule = ground_contact1)

def ground_contact2(m,n):
    if n < N:
        return m.ground_penalty2[n,'contact'] == m.foot2_p[n+1,'y']*m.GRF2[n,'y','ps'] 
        # notice that the GRF is complemented with the foot height at the NEXT node
    else:
        return Constraint.Skip
m.ground_contact2 = Constraint(m.N, rule = ground_contact2)

# friction
def ground_friction1(m,n):
    return m.ground_penalty1[n,'friction'] == (m.foot1_v[n,'x','ps']+m.foot1_v[n,'x','ng'])*m.friction_cone1[n]
m.ground_friction1 = Constraint(m.N, rule = ground_friction1)

def ground_friction2(m,n):
    return m.ground_penalty2[n,'friction'] == (m.foot2_v[n,'x','ps']+m.foot2_v[n,'x','ng'])*m.friction_cone2[n]
m.ground_friction2 = Constraint(m.N, rule = ground_friction2)

# slipping
def ground_slip1_ps(m,n):
    return m.ground_penalty1[n,'slip_ps'] == m.foot1_v[n,'x','ps']*m.GRF1[n,'x','ps']
m.ground_slip1_ps = Constraint(m.N, rule = ground_slip1_ps)

def ground_slip2_ng(m,n):
    return m.ground_penalty2[n,'slip_ng'] == m.foot2_v[n,'x','ng']*m.GRF2[n,'x','ng']
m.ground_slip2_ng = Constraint(m.N, rule = ground_slip2_ng)

# bound contact forces at first node
for dof in DOFs:
    for sgn in signs:
        m.GRF1[N,dof,sgn].value = 0
        m.GRF1[N,dof,sgn].fixed = True
        m.GRF2[N,dof,sgn].value = 0
        m.GRF2[N,dof,sgn].fixed = True
        
print("Done")

Done


In [ ]:
# ROTARY JOINT --------------------------------------------------------------------------------------------------------------
# the model's "hip"
# points of interaction: 1) the body COM 2) the top of leg link 1

# sets
joints = ['hip','knee']
m.J = Set(initialize = joints)

joint_constraints = ['up','lo'] # set of joint penalties
m.joint_constraints = Set(initialize = joint_constraints)

# parameters
hip_bound = [-np.pi/2,np.pi/2]
m.hip_bound = Param(m.joint_constraints, initialize = {'up':hip_bound[1],'lo':hip_bound[0]}) 
knee_bound = [-np.pi/2,np.pi/2]
m.knee_bound = Param(m.joint_constraints, initialize = {'up':knee_bound[1],'lo':knee_bound[0]}) 

# variables
m.top = Var(m.N, m.DOF) # we only need the top of leg link 1

m.th_joint = Var(m.N, bounds = (hip_bound[0],hip_bound[1])) 

m.tau_r = Var(m.N, m.sgn, bounds = (0.0,None)) # rebound torque

tau_lim = 2
m.tau_a = Var(m.N, bounds = (-tau_lim,tau_lim)) # actuator torque

m.joint_penalty = Var(m.N, m.J, m.joint_constraints, bounds = (0.0,None))

m.F_c = Var(m.N, m.DOF) # constraint forces 

# constraints

# aux variables
def def_top(m,n,dof):
    if dof == 'x':
        return m.top[n,dof] == m.X[n,('leg',1),'x'] - 0.5*m.len[('leg',1)]*sin(m.X[n,('leg',1),'theta'])
    if dof == 'y':
        return m.top[n,dof] == m.X[n,('leg',1),'y'] + 0.5*m.len[('leg',1)]*cos(m.X[n,('leg',1),'theta'])
    else:
        return Constraint.Skip
m.def_top = Constraint(m.N, m.DOF, rule = def_top)
    
def def_th_joint(m,n):
    return m.th_joint[n] == m.X[n,('leg',1),'theta'] - m.X[n,('body',1),'theta']
m.def_th_joint = Constraint(m.N, rule = def_th_joint)

# connection
def connect_hip(m,n,dof):
    if dof == 'theta': 
        return Constraint.Skip
    else:
        return m.top[n,dof] == m.X[n,('body',1),dof]
m.connect_hip = Constraint(m.N, m.DOF, rule = connect_hip)

# complementarity
def hip_limits(m,n,jc):
    if n < N:
        if jc == 'up':
            # NEXT angle
            return m.joint_penalty[n,'hip',jc] == (m.hip_bound['up'] - m.th_joint[n+1])*m.tau_r[n,'ng']
        else:
            return m.joint_penalty[n,'hip',jc] == (m.th_joint[n+1] - m.hip_bound['lo'])*m.tau_r[n,'ps']
    else:
        return Constraint.Skip
m.hip_limits = Constraint(m.N, m.joint_constraints, rule = hip_limits)

# bound contact forces at last node
for sgn in signs:
    m.tau_r[N,sgn].value = 0
    m.tau_r[N,sgn].fixed = True